In [6]:
!ls

first_tweet.ipynb


In [3]:
!pip install -r /mnt/nfs/homes/athirion/git/SentimentalBB/requirements.txt

In [2]:
! pip list

Package                Version   Editable project location
---------------------- --------- -----------------------------------------
aiohttp                3.8.1
aiohttp-retry          2.4.6
aiosignal              1.2.0
antlr4-python3-runtime 4.8
anyio                  3.5.0
appdirs                1.4.4
argon2-cffi            21.3.0
argon2-cffi-bindings   21.2.0
asgiref                3.5.0
asttokens              2.0.5
async-timeout          4.0.2
asyncssh               2.8.1
atpublic               3.0.1
attrs                  21.4.0
autopep8               1.6.0
awscli                 1.22.68
backcall               0.2.0
bleach                 4.1.0
botocore               1.24.13
certifi                2021.10.8
cffi                   1.15.0
charset-normalizer     2.0.12
click                  8.0.4
colorama               0.4.3
colorlog               6.6.0
commonmark             0.9.1
configobj              5.0.6
cryptography           36.0.1
debugpy                1.5.1
decorator    

In [3]:
import searchtweets

searchtweets.__version__

ModuleNotFoundError: No module named 'searchtweets'

In [ ]:
from searchtweets import ResultStream, gen_request_parameters, load_credentials

In [ ]:
search_args = load_credentials(filename="./search_tweets_creds_example.yaml",
                 yaml_key="search_tweets_v2_example",
                 env_overwrite=False)

In [ ]:
search_args = load_credentials(filename=None)


In [ ]:
# search_args

In [ ]:
query = gen_request_parameters("Emmanuel Macron", results_per_call=100, granularity=None)
print(query)

In [ ]:
from searchtweets import collect_results

In [ ]:
tweets = collect_results(query, max_tweets=100, result_stream_args=search_args) 

In [ ]:
tweets

In [ ]:
type(tweets)

In [ ]:
func = lambda tweets, tweet_nb: tweets[0]['data'][tweet_nb]['text']

In [ ]:
print(func(tweets, 11)[::])

In [ ]:

request = tweets
for element in request:
    for id, tweet in enumerate(element['data']):
        print(f"Tweet n*{id}")
        print(tweet["text"])
        print(f"")


In [ ]:
[print(tweet["text"], end='\n\n') for tweet in tweets[0:10]]